# Welcome to SUPRA on Jupyter
This document contains the process of using Intel(R) oneAPI Base Toolkit build and run SUPRA on Intel GPU.
(This file contains all commands you need to run SUPRA on GPU)

**Intel(R) oneAPI is still in beta phase, this project only for test and should not be used in products.** 

## Table of Contents
1. [SUPRA introduction](#sec-supra)
2. [Build steps](#sec-steps)
3. [Performance](#sec-perf)

<a id="sec-supra"></a>
## 1. SUPRA introduction

The SUPRA is an open source project, your can find detailed information at <a href="https://github.com/IFL-CAMP/supra">SUPRA</a>.

<a id="sec-steps"></a>
## 2. Build steps

#### **Usage Tip:**
**Create an empty folder in your linux machine(ubuntu 18.04).Copy this file to the folder. launch jupyter lab or jupyter notebook and open this file, follow the below command.**

### 2.1 Download source code from github and apply patch

In [ ]:
%%bash
# Download source code and patches
git clone https://github.com/IFL-CAMP/supra.git
git clone https://github.com/intel/supra-on-oneapi.git
    
cd supra 
git reset --hard 73c930a08a7b1087f5be588863876a648a1add99 
git am --whitespace=nowarn ../supra-on-oneapi/*.patch 
git am --whitespace=nowarn ../supra-on-oneapi/supra-jupyter/*.patch

### 2.1 Download Ultrasound data and unzip files

In [ ]:
%%bash
echo "Downloading raw ultrasound data,you may need wait minutes..."
wget http://campar.in.tum.de/files/goeblr/mockData_linearProbe.zip -o /dev/null
wget https://github.com/IFL-CAMP/supra/raw/master/config/configDemo.xml -o /dev/null

mkdir data

mv configDemo.xml data
mv mockData_linearProbe.zip data

cd data && unzip mockData_linearProbe.zip -d . && rm mockData_linearProbe.zip

echo "data ok!"

### Note: If import wget fails, run below command to install wget and re-run above cell

In [ ]:
!pip install wget

### Use below command in your terminal to install Below 3rd libraries if you first run SUPRA on your machine.

sudo apt-get install cmake cmake-gui qt5-default libtbb-dev libopenigtlink-dev git

#### Build project for GPU

In [ ]:
%%bash
# Setup oneAPI environment
echo "Build process stat, this may take minutes"
source /opt/intel/inteloneapi/setvars.sh --force
set -x
dpcpp --version

echo "Remove old build folder..."
cd supra && rm -rf build && mkdir -p build && cd build
echo "Remove done."


export PATH=`echo ${PATH} | awk -v RS=: -v ORS=: '/vtune/ {next} {print}'`

CC=clang CXX=dpcpp cmake .. 
make -j4 &> /dev/null
echo "Build done!"

#### Run on GPU in Commandline version
After run a while, you can interrupt below cell to stop supra running.

In [ ]:
%%bash
# Run SUPRA on GPU
source /opt/intel/inteloneapi/setvars.sh --force

cd supra/build
cp -r ../../data .
rm -rf RxBeamformer HilbertFirEnvelope LogCompressor ScanConverter
rm supra.log
mkdir RxBeamformer HilbertFirEnvelope LogCompressor ScanConverter

./src/CommandlineInterface/SUPRA_CMD data/configDemo.xml

#### ====== Note: Run on GPU in Graphic version =======

#### Open a terminal, In supra/build foler, run below command to see output.

source /opt/intel/inteloneapi/setvars.sh

src/GraphicInterface/SUPRA_GUI -c data/configDemo.xml -a


#### ====== End of Note =======

#### Display GPU output

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# image path
fpga_img_path = ["./supra/build/RxBeamformer/", "./supra/build/HilbertFirEnvelope/", 
               "./supra/build/LogCompressor/", "./supra/build/ScanConverter/"]
col_labels = ["RxBeamformer", "HilbertFirEnvelope", "LogCompressor", "ScanConverter"]
row_labels = ["Frame #" + str(i+1) for i in range(8)]

for row_img_index in range(8):
        
    fig, axs = plt.subplots(1, 4, figsize=(10,5))
    
    if row_img_index == 0:
        for i, col in zip(range(4), col_labels):
            axs[i].set_title(col)
        
    for col_img_index in range(4):
        current_img_path = fpga_img_path[col_img_index] + str(row_img_index + 1) + '.png'
        current_img = plt.imread(current_img_path)
        axs[col_img_index].imshow(current_img, "gray")
        axs[col_img_index].axis('off')
    fig.suptitle(row_labels[row_img_index], x= -0.05, y=0.5,fontsize = 9)
    plt.tight_layout()
    plt.show()
plt.show()

<a id="sec-perf"></a>
## 3. Performance

The performance data were written to  ./supra/build/supra.log file. run below cell to check.

In [ ]:
!cat ./supra/build/supra.log